In [2]:
import config
import code_improvement.code_execution as execution

from utils import prompt, tools
from langchain.prompts import ChatPromptTemplate

In [3]:
def determine_term_list():
    term_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a very helpful assistant."),
        ("user", "{input}")
    ])
    term_chain = term_prompt | config.llm
    
    response = term_chain.invoke({
        "input": prompt.concept_determination_prompt.format(config.user_input)
    })
    term_list = response.content.split(', ')
    
    return term_list

In [4]:
def determine_URL_list(term_list):
    URL_list = []
    agent_executor = tools.create_agent(have_message_history=False, tool_list=[config.search], message_history=None)
    
    for term in term_list:
        print("Working on term: {} . . .".format(term))
        
        response = agent_executor.invoke({
            "input": prompt.concept_searching_prompt.format(config.user_input, term)
        })
        
        URL_list.extend(response['output'].split(', '))
                
    return URL_list

In [5]:
def generate_high_level_design(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.high_level_design_prompt.format(config.user_input)},
        config=config.session_configuration
    )
    
    return response['output']

In [6]:
def generate_detailed_design(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.detailed_design_prompt.format(config.user_input)},
        config=config.session_configuration
    )
    
    step_list = response['output'].split("\n\n")
    
    return step_list

In [7]:
def generate_code_segment(agent_executor, step_list):
    code_list = []
    
    for step_description in step_list:
        print('Now, we are generating code segment for step {}'.format(
            step_description.split('\n')[0]
        ))
        
        try:
            response = agent_executor.invoke(
                {"input": prompt.code_segment_prompt.format(
                    config.user_input, 
                    config.programming_language,
                    step_description.split('\n')[0],
                    step_description[step_description.index('\n') + 1:],
                    config.programming_language,
                    step_description.split('\n')[0],
                )},
                config=config.session_configuration
            )
        except:
            continue
        
        code_segment = tools.sanitize_output(response['output'])
        code_list.append(code_segment)
        
    return code_list

In [8]:
def generate_final_code(agent_executor):
    response = agent_executor.invoke(
        {"input": prompt.code_combination_prompt.format(config.user_input)},
        config=config.session_configuration
    )
    
    return tools.sanitize_output(response['output'])

In [9]:
def execute_code(code, data_path):
    with open('generated_code.py', 'w') as file:
        file.write(code)
        
    command = "python3 generated_code.py -i {}".format(data_path)
    status, program_output = execution.run(command, True)
    
    return status, program_output

In [9]:
# first determine a list of terms to be serached
term_list = determine_term_list()
print("According to your requirements, we decide to search for the definition of the following terms:", term_list)

According to your requirements, we decide to search for the definition of the following terms: ['XRF55 dataset', 'heatmaps', 'mmWave radars', 'human-computer interaction motions', 'human motion recognition system', 'load', 'preprocess', 'machine learning model', 'recognition accuracy', 'test dataset', 'PyTorch', 'GPU', 'npy files', 'user_id', 'activity_id', 'trial_number', 'Human-Computer Interaction.']


In [10]:
URL_list = determine_URL_list(term_list)
print("Here is the URL list we crawled:")
for URL in URL_list:
    print(URL)

Working on term: XRF55 dataset . . .


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'XRF55 dataset definition'}`


[{'url': 'https://github.com/aiotgroup/XRF55-repo/blob/main/README.md', 'content': 'To facilitate the transitioning, in this paper, we introduce and release a large-scale multiple radio frequency dataset, named XRF55, for indoor human action analysis. XRF55 encompasses 42.9K RF samples and 55 action classes of human-object interactions, human-human interactions, fitness, body motions, and human-computer interactions, collected ...'}, {'url': 'https://github.com/aiotgroup/XRF55-repo', 'content': 'To facilitate the transitioning, in this paper, we introduce and release a large-scale multiple radio frequency dataset, named XRF55, for indoor human action analysis. XRF55 encompasses 42.9K RF samples and 55 action classes of human-object interactions, human-human interactions, fitness, body motions, and human-computer interactions

In [16]:
while input("Do you want to provide extra URLs? Y/N") == 'Y':
    URL_list.append(input("Please input the extra URL here: "))
    
print("The updated URL_list is:")
for URL in URL_list:
    print(URL)

The updated URL_list is:
https://github.com/aiotgroup/XRF55-repo/blob/main/README.md
http://13.229.71.32/
https://en.wikipedia.org/wiki/Heat_map
https://www.ti.com/lit/SPYY005
https://dev.ti.com/tirex/explore/node?node=A__AXNV8Pc8F7j2TwsB7QnTDw__RADAR-ACADEMY__GwxShWe__LATEST
https://www.ti.com/sensors/mmwave-radar/overview.html
https://www.techtarget.com/searchnetworking/definition/millimeter-wave-MM-wave
https://www.utmel.com/blog/categories/sensors/millimeter-wave-radar-advantages-types-and-applications
https://www.sciencedirect.com/topics/psychology/human-computer-interaction
https://www.interaction-design.org/literature/topics/human-computer-interaction
https://link.springer.com/referenceworkentry/10.1007/978-0-387-39940-9_192
https://en.wikipedia.org/wiki/Human–computer_interaction
https://www.frontiersin.org/articles/10.3389/fnbot.2022.860981/full
https://arxiv.org/pdf/1608.06761
https://www.hindawi.com/journals/complexity/2021/6679746/
https://journals.sagepub.com/doi/full/10.1

In [10]:
URL_list = [
    'https://github.com/aiotgroup/XRF55-repo/blob/main/README.md',
    'http://13.229.71.32/',
    'https://en.wikipedia.org/wiki/Heat_map',
    'https://www.ti.com/lit/SPYY005',
    'https://dev.ti.com/tirex/explore/node?node=A__AXNV8Pc8F7j2TwsB7QnTDw__RADAR-ACADEMY__GwxShWe__LATEST',
    'https://www.ti.com/sensors/mmwave-radar/overview.html',
    'https://www.techtarget.com/searchnetworking/definition/millimeter-wave-MM-wave',
    'https://www.utmel.com/blog/categories/sensors/millimeter-wave-radar-advantages-types-and-applications',
    'https://www.sciencedirect.com/topics/psychology/human-computer-interaction',
    'https://www.interaction-design.org/literature/topics/human-computer-interaction',
    'https://link.springer.com/referenceworkentry/10.1007/978-0-387-39940-9_192',
    'https://en.wikipedia.org/wiki/Human–computer_interaction',
    'https://www.frontiersin.org/articles/10.3389/fnbot.2022.860981/full',
    'https://arxiv.org/pdf/1608.06761',
    'https://www.hindawi.com/journals/complexity/2021/6679746/',
    'https://journals.sagepub.com/doi/full/10.1177/1729881420983219',
    'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9113890/',
    'https://www.dictionary.com/browse/load',
    'https://www.thefreedictionary.com/load',
    'https://dictionary.cambridge.org/us/dictionary/english/load',
    'https://dictionary.cambridge.org/dictionary/english/load',
    'https://www.merriam-webster.com/dictionary/load',
    'https://www.collinsdictionary.com/dictionary/english/preprocess',
    'https://en.wikipedia.org/wiki/Data_Preprocessing',
    'https://www.techtarget.com/searchdatamanagement/definition/data-preprocessing',
    'https://www.merriam-webster.com/dictionary/preprocess',
    'https://en.wikipedia.org/wiki/Preprocessor',
    'https://en.wikipedia.org/wiki/Machine_learning',
    'https://simple.wikipedia.org/wiki/Machine_learning',
    'https://en.wikipedia.org/wiki/Accuracy_and_precision',
    'https://dictionary.cambridge.org/example/english/recognition-accuracy',
    'https://en.wikipedia.org/wiki/Training,_validation,_and_test_data_sets',
    'https://en.wikipedia.org/wiki/PyTorch',
    'https://en.wikipedia.org/wiki/Graphics_processing_unit',
    'https://note.nkmk.me/en/python-numpy-load-save-savez-npy-npz/',
    'https://fileinfo.com/extension/npy',
    'https://numpy.org/doc/stable/user/absolute_beginners.html',
    'https://techterms.com/definition/user_id',
    'https://en.wikipedia.org/wiki/User_identifier',
    'https://www.indicative.com/resource/user-identification-user/',
    'https://www.techopedia.com/definition/12552/user-identification-user-id',
    'https://learn.microsoft.com/en-us/dotnet/api/system.diagnostics.activity.id?view=net-8.0',
    'https://learn.microsoft.com/en-us/dotnet/core/diagnostics/eventsource-activity-ids',
    'https://xapi.com/blog/deep-dive-activity/',
    'https://www.ibm.com/docs/SSRMWJ_7.0.1/com.ibm.isim.doc/dbschema/ref/im51_dbschema_activitytable.htm',
    'https://docs.temporal.io/activities',
    'https://www.who.int/clinical-trials-registry-platform/unambiguous-trial-identification/the-universal-trial-number-(utn)',
    'https://simple.wikipedia.org/wiki/Human–computer_interaction',
    'https://en.wikipedia.org/wiki/Human–computer_interaction',
    'https://aiotgroup.github.io/XRF55/'
]

In [11]:
additional_information = """01, Carrying Weight
02, Brushing Floor
03, Cutting
04, Wearing a Hat
05, Use the Phone
06, Throw Something
07, Put Something on the Table
08, Put on Clothing
09, Picking
10, Drinking
11, Smoking
12, Eating
13, Brushing Teeth
14, Blow Dry Hair
15, Brush Hair
16, Shake Hands
17, Hugging
18, Hand Something to Someone
19, Kick Someone
20, Hit Someone with Something
21, Choke Someone's Neck
22, Push Someone
23, Body Weight Squats
24, Tai Chi
25, Boxing
26, Weightlifting
27, Hula Hooping
28, Jump Rope
29, Jumping Jack
30, High Leg Lift
31, Waving
32, Clap Hands
33, Fall on the Floor
34, Jumping
35, Running
36, Sitting Down
37, Standing Up
38, Turning
39, Walking
40, Stretch Oneself
41, Pat on Shoulder
42, Playing Erhu
43, Playing Ukulele
44, Playing Drum
45, Foot Stamping
46, Shaking Head
47, Nodding
48, Drawing a Circle
49, Drawing a Cross
50, Pushing
51, Pulling
52, Swiping Left
53, Swiping Right
54, Swiping Up
55, Swiping Down
"""

In [12]:
from langchain_core.documents import Document

document_list = tools.load_new_webpage(URL_list)
from langchain_core.documents import Document
document_list.append(Document(page_content=additional_information))

We will further search information from these websites:
https://github.com/aiotgroup/XRF55-repo/blob/main/README.md
http://13.229.71.32/
https://en.wikipedia.org/wiki/Heat_map
https://www.ti.com/lit/SPYY005
https://dev.ti.com/tirex/explore/node?node=A__AXNV8Pc8F7j2TwsB7QnTDw__RADAR-ACADEMY__GwxShWe__LATEST
https://www.ti.com/sensors/mmwave-radar/overview.html
https://www.techtarget.com/searchnetworking/definition/millimeter-wave-MM-wave
https://www.utmel.com/blog/categories/sensors/millimeter-wave-radar-advantages-types-and-applications
https://www.sciencedirect.com/topics/psychology/human-computer-interaction
https://www.interaction-design.org/literature/topics/human-computer-interaction
https://link.springer.com/referenceworkentry/10.1007/978-0-387-39940-9_192
https://en.wikipedia.org/wiki/Human–computer_interaction
https://www.frontiersin.org/articles/10.3389/fnbot.2022.860981/full
https://arxiv.org/pdf/1608.06761
https://www.hindawi.com/journals/complexity/2021/6679746/
https://jou

In [13]:

agent_with_chat_history = tools.create_agent(
    have_message_history=True,
    tool_list=[config.search, tools.create_context_retrieval_tool(document_list)],
    message_history=config.message_history
)

In [16]:
high_level_design = generate_high_level_design(agent_with_chat_history)
print("We first provide the algorithm design outline:\n{}".format(high_level_design))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'pytorch human motion recognition heatmap'}`


[{'url': 'https://www.nature.com/articles/s41598-023-45149-5', 'content': 'PyTorch 11.3 is used to build a DL model. ... The red regions on the heatmap indicate the regions that the network prioritizes. ... M. G. Radar-based human-motion recognition with deep learning ...'}, {'url': 'https://learnopencv.com/human-pose-estimation-using-keypoint-rcnn-in-pytorch/', 'content': 'Human Pose Estimation is an important research area in the field of Computer Vision. It deals with estimating unique points on the human body, also called keypoints. In this blog post, we will discuss one such algorithm for finding keypoints on images containing a human called Keypoint-RCNN. The code is written in Pytorch, using the Torchvision library.'}, {'url': 'https://github.com/GuyTevet/motion-diffusion-model', 'content': '--motion_length (text-to-motion only) in second

In [17]:
detailed_design_step_list = generate_detailed_design(agent_with_chat_history)

print("Based on the high level design, we provide the detailed design as:")
for element in detailed_design_step_list:
    print(element, '\n')



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'load and preprocess npy files for deep learning'}`


[{'url': 'https://towardsdatascience.com/introduction-to-keras-part-two-data-preprocessing-e1377d09fac', 'content': 'First things first. Fetch and download the IMDb data (or any text dataset) from an available data source. Convert the given text directory into tf.data.Dataset by using the method text_dataset_from_directory (). batch_size = 32. seed = 42 raw_train_ds = keras.preprocessing.text_dataset_from_directory (.'}, {'url': 'https://stackoverflow.com/questions/53788434/training-a-keras-model-from-batches-of-npy-files-using-generator', 'content': 'So here generate_batch_from_directory() should take image_file_i.npy and label_file_i.npy every time and optimise the weight until there is no batch left. Each image array in the .npy files has already been processed with augmentation, rotation and scaling. Each .npy file is properly mixed w

In [18]:
code_segment_list = generate_code_segment(
    agent_with_chat_history, detailed_design_step_list
)

Now, we are generating code segment for step Step 1: Load and preprocess the XRF55 dataset


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Python code to recursively read .npy files from subfolders'}`


[{'url': 'https://www.sharpsightlabs.com/blog/numpy-load/', 'content': "In the simplest case, the syntax for Numpy load is simple. Assuming that you've imported Numpy with the alias np, you type the function as np.load (). Inside the parenthesis, you provide the name of the .npy or .npz file that you want to load. The first argument to the function is the name of the file from which you want to load your data."}, {'url': 'https://stackoverflow.com/questions/2212643/python-recursive-folder-read', 'content': 'I am writing a script to recursively read the contents of text files in a folder structure. The problem I have is the code I have written will only work for one folder deep. ... Python Code: import os import sys rootdir = sys.argv[1] f

In [19]:
final_code = generate_final_code(agent_with_chat_history)



> Entering new AgentExecutor chain...
```python
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import argparse

# Custom Dataset class for XRF55 dataset to work with PyTorch DataLoader
class XRF55Dataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor, label_tensor: torch.Tensor):
        self.data_tensor = data_tensor
        self.label_tensor = label_tensor

    def __len__(self):
        return len(self.data_tensor)

    def __getitem__(self, index):
        return self.data_tensor[index], self.label_tensor[index]

# Custom CNN architecture for human motion recognition
class CustomCNN(nn.Module):
    def __init__(self, num_classes=11):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=Tru

In [20]:
dataset_path = input("Please input the path to your dataset: ")
# dataset_path = "/home/iot/Documents/data/mmWave"

In [21]:
print("Next, we will proceed to the supervised code generation and modification stage.")

for epoch in range(config.epoch_number):
    status, program_output = execute_code(final_code, dataset_path)
    while status != 0:
        response = agent_with_chat_history.invoke(
            {"input": prompt.correct_grammar_prompt.format(config.user_input, program_output)},
            config=config.session_configuration
        )
        final_code = tools.sanitize_output(response['output'])
        
        status, program_output = execute_code(final_code, dataset_path)

    response = agent_with_chat_history.invoke(
        {"input": prompt.improve_code_prompt.format(config.user_input, program_output)},
        config=config.session_configuration
    )
    high_level_design = response['output']
    detailed_design_step_list = generate_detailed_design(agent_with_chat_history)
    code_segment_list = generate_code_segment(
        agent_with_chat_history, detailed_design_step_list
    )
    final_code = generate_final_code(agent_with_chat_history)



Next, we will proceed to the supervised code generation and modification stage.
************** START **************
Traceback (most recent call last):
File "/home/iot/Documents/AutoNLP/generated_code.py", line 143, in <module>
main(args.input)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 127, in main
data_tensor, label_tensor = load_and_preprocess_xrf55_dataset(dataset_folder_path)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 78, in load_and_preprocess_xrf55_dataset
data_tensor = torch.stack(data_list)
RuntimeError: stack expects a non-empty TensorList
************** FAILED **************


> Entering new AgentExecutor chain...
```python
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import argparse

# Custom Dataset class for XRF55 dataset to work with PyTorch DataLoader
class XRF55Dataset(Dataset):
    def __init__(sel

In [36]:
# search for all the generated comprehensive code
combined_code_list = []

for index in range(0, len(config.message_history.messages), 2):
    question = config.message_history.messages[index].content
    
    if "Based on the user's problem and all the generated code segments for each module, please constructively integrate them to form a comprehensive and cohesive piece of code. Please also embed detailed comments." in question:
        combined_code = tools.sanitize_output(
            config.message_history.messages[index + 1].content
        )
        combined_code_list.append(combined_code)

In [46]:
program_output_list = []

for index in range(len(combined_code_list)):
    status, program_output = execute_code(combined_code_list[index], dataset_path)
    program_output_list.append(program_output)

************** START **************
Traceback (most recent call last):
File "/home/iot/Documents/AutoNLP/generated_code.py", line 151, in <module>
main(args.input)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 142, in main
train_model(model, train_loader)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 110, in train_model
outputs = model(inputs)
File "/home/iot/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
return forward_call(*input, **kwargs)
File "/home/iot/Documents/AutoNLP/generated_code.py", line 52, in forward
x = self.features(x)
File "/home/iot/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
return forward_call(*input, **kwargs)
File "/home/iot/.local/lib/python3.10/site-packages/torch/nn/modules/container.py", line 204, in forward
input = module(input)
File "/home/iot/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
return forwa

In [47]:
running_result = ""

for index in range(len(program_output_list)):
    running_result = "{}\n\n<version_{}>\n{}\n</version_{}>\n\n".format(
        running_result, index + 1, program_output_list[index], index + 1
    )


response = agent_with_chat_history.invoke(
    {"input": prompt.final_decision_prompt.format(running_result)},
    config=config.session_configuration
)



> Entering new AgentExecutor chain...
3

> Finished chain.


In [48]:
version_index = int(response['output'])
final_code = combined_code_list[version_index - 1]

print("Here is the final version of code to solve your problem.")
print("============================================")
print(final_code)
print("============================================")

Here is the final version of code to solve your problem.
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import argparse

# Custom Dataset class for XRF55 dataset to work with PyTorch DataLoader
class XRF55Dataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor, label_tensor: torch.Tensor):
        self.data_tensor = data_tensor
        self.label_tensor = label_tensor

    def __len__(self):
        return len(self.data_tensor)

    def __getitem__(self, index):
        return self.data_tensor[index], self.label_tensor[index]

# Custom CNN architecture for human motion recognition
class CustomCNN(nn.Module):
    def __init__(self, num_classes=11):
        super(CustomCNN, self).__init__()
        self.featu

In [49]:
response = agent_with_chat_history.invoke(
    {"input": prompt.user_documentation_prompt.format(config.user_input, final_code)},
    config=config.session_configuration
)



> Entering new AgentExecutor chain...
# User Documentation for Human Motion Recognition System

## Introduction

The Human Motion Recognition System is designed to process and analyze heatmaps extracted from mmWave radars to classify various human-computer interaction motions. The system leverages a deep learning model to achieve this goal, with the PyTorch framework being the preferred choice due to its flexibility and support for GPU acceleration.

The XRF55 dataset, which the system utilizes, comprises mmWave radar-generated heatmaps that capture distinct human motions. The dataset is structured into four sub-folders, each containing numerous `.npy` files representing different activity trials by various users. Each `.npy` file follows the naming convention `<user_id>_<activity_id>_<trial_number>.npy` and contains a heatmap with the shape `(1, 17, 256, 128)`.

The primary objective is to train a machine learning model that can accurately classify 11 specified activities within the